## spacy基础

In [4]:
import spacy 

nlp = spacy.blank('zh') #选择nlp的语言 #zh代表中文

- token.text 可以用来输出文本内容

In [2]:
doc = nlp("This is a sentence.") # nlp的内容转化为token

for token in doc:
    print(token.text)

This
is
a
sentence
.


- doc[i]可以读取指定的token

In [5]:
# 输出第一个token

first_token = doc[0]

print("the first token in documents:", first_token.text)

the first token in documents: This


In [6]:
# 输出一串token

slice_token = doc[1:3]
print(slice_token.text)

is a


- like_num 可以用来判断是否是数字
- token.i 可以指示当前token的index

In [10]:
doc1 = nlp(
    "In 1990, more than 60% of people in East Asia were in extreme poverty. "
    "Now less than 4% are."
)

for token in doc1:
    # check  if the token is a number
    if token.like_num:
        # get the next token 
        next_token = doc1[token.i+1]
        # check is the next token is %
        if next_token.text == '%':
            print("percent found", token.text)

percent found 60
percent found 4


## Trained pipelines 基础

### 什么是Trained pipelines？
能够在上下文中预测语言属性的模型:包括词性标签、句法依赖和命名实体。
eg:一个词是动词还是一段文本是否是人名

## piprlines 包
使用该spacy download命令下载。

#### “zh_core_web_sm”包是一个小型中文管道，函数后面都有下划线
- .pos_属性，是词性标注的结果   
- .dep_属性返回预测的依存关系标注。
- .head属性返回句法头词符。你可以认为这是词在句子中所依附的母词符。

#### 例子：我吃了个肉夹馍
label1： nsubj， 是名词主语， 我就是依附在吃上面的nsubj
label2: dobj， 是目的语，肉夹馍是依附在吃上面的dobj

- doc.ents中可以读取命名实体识别模型预测出的所有命名实体。
- 其中.label_ 属性来打印出实体标注。
- spacy.explain 可以解释label

In [23]:
nlp = spacy.load("zh_core_web_sm")

In [9]:
doc = nlp("微软准备用十亿美金买下英国的创业公司")

for ent in doc.ents:
    print(ent.text, ent.label_)
    
spacy.explain("ORG")

微软 ORG
十亿美金 MONEY
英国 GPE


'Companies, agencies, institutions, etc.'

In [10]:
print(doc.text)

微软准备用十亿美金买下英国的创业公司


In [12]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

微软 PROPN nsubj
准备 VERB ROOT
用 ADP case
十亿 NUM nmod:prep
美金 NUM mark:clf
买下 VERB ccomp
英国 PROPN dobj
的 PART mark
创业 NOUN compound:nn
公司 NOUN dobj


## matcher
写一些规则来寻找文本中的目标词汇和短语。
- 与正则表达式相比，matcher是配合Doc和Token这样的方法来使用的， 而不是只作用于字符串上。
- 匹配的模板是一些列表，列表的每一个元素是一个字典。 每个字典代表一个词符，键值是词符属性名，映射到对应的目标值上面。

- nlp.vocab来初始化matcher
- matcher.add方法可以用来添加一个模板。第一个参数是唯一的ID用来识别匹配的是哪一个模板。 第二个参数是一个模板的列表。

- 当你对doc调用matcher时会返回一个列表，列表中的每个元素是一个元组(tuple)。
- 每个元组由三个值构成：匹配到的ID，匹配到的跨度的起始和终止索引



In [13]:
from spacy.matcher import Matcher

In [24]:
# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 用模型分享出的vocab初始化matcher
matcher = Matcher(nlp.vocab)

# 给matcher加入模板
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]
matcher.add("IPHONE_PATTERN", [pattern])

# 处理文本
doc = nlp("即将上市的iPhone X发布日期被泄露了")

# 在doc上面调用matcher
matches = matcher(doc)

# 遍历所有的匹配结果
for match_id, start, end in matches:
    # 获取匹配的跨度
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


In [21]:
pattern = [
    {"IS_DIGIT": True},
    {"LOWER": "国际"},
    {"LOWER": "足联"},
    {"LOWER": "世界杯"},
    {"IS_PUNCT": True}
]
doc = nlp("2018国际足联世界杯：法国队赢了！")

In [29]:
# 一个词根是"喜欢"的动词，后面跟着一个名词。
pattern1 = [
    {"LEMMA": "喜欢", "POS": "VERB"},
    {"POS": "NOUN"}
]
doc = nlp("我喜欢狗但我更喜欢猫。")

In [30]:
# 使用运算符和量词来定义一个词符应该被匹配几次。 我们可以用"OP"这个关键词来添加它们。
pattern = [
    {"LEMMA": "买"},
    {"POS": "NUM", "OP": "?"},  # 可选: 匹配0次或者1次
    {"POS": "NOUN"}
]
doc = nlp("我买个肉夹馍。我还要买凉皮。")

- {"OP": "!"}	否定: 0次匹配
- {"OP": "?"}	可选: 0次或1次匹配
- {"OP": "+"}	1次或更多次匹配
- {"OP": "*"}	0次或更多次匹配

In [32]:
# 写一个模板，只匹配到所有提及完整iOS版本的部分： “iOS 7”，“iOS 11”和”iOS 10”。
import spacy

from spacy.matcher import Matcher
nlp = spacy.load("zh_core_web_sm")

matcher = Matcher(nlp.vocab)

doc = nlp(
    "升级iOS之后，我们并没有发现系统设计有很大的不同，远没有当年iOS 7发布时带来的"
    "焕然一新的感觉。大部分iOS 11的设计与iOS 10保持一致。但我们仔细试用后也发现了一些"
    "小的改进。"
)

# 写一个模板来匹配完整的iOS版本 ("iOS 7", "iOS 11", "iOS 10")
pattern = [{"TEXT":"iOS"}, {"IS_DIGIT": True}]

matcher.add("IOS_VERSION_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# 遍历所有的匹配，然后打印span的文本
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 3
Match found: iOS 7
Match found: iOS 11
Match found: iOS 10


In [36]:
# 写一个模板，只匹配到不同格式的”download”词（词符的原词是”download”）
# 后面跟着一个词性是"PROPN"（专有名词）的词符。

import spacy
from spacy.matcher import Matcher

nlp = spacy.load("zh_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "我之前有去下载Dota到电脑上面，但是根本打不开游戏，怎么办？"
    "我下载Minecraft，是Windows的版本，下载后是一个'.zip'的文件夹，然后我用了默认软件做了"
    "解压...我是不是还需要去下载Winzip？"
)

# 写一个模板来匹配"下载"加一个代词
pattern = [{"TEXT": "下载"}, {"POS": "PROPN"}]

# 把模板加入到matcher中，然后把matcher应用到doc上面
matcher.add("DOWNLOAD_THINGS_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# 遍历所有的匹配，打印span的文本
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 2
Match found: 下载Dota
Match found: 下载Minecraft


In [38]:
# 写一个模板，匹配到形容词（"ADJ"） 
# 后面跟着一两个名词"NOUN"（一个名词和另一个可能有的名词）。

import spacy
from spacy.matcher import Matcher

nlp = spacy.load("zh_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "这个app的特性包括了优雅设计、快捷搜索、自动标签以及可选声音。"
)

pattern = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN", "OP": "?"}]

matcher.add("ADJ_NOUN_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# 遍历所有的匹配，打印span的文本
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 4
Match found: 优雅设计
Match found: 快捷搜索
Match found: 自动标签
Match found: 可选声音
